In [1]:
import os
import sys
from psycopg2 import pool
from dotenv import load_dotenv
import pandas as pd
from asgiref.sync import sync_to_async

In [2]:
import django
from django.core.management import execute_from_command_line

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "app.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from pets.models import PetRace

# test.objects.all().delete()

In [3]:
execute_from_command_line(['manage.py', 'showmigrations'])

In [4]:
execute_from_command_line(['manage.py', 'makemigrations'])

Migrations for 'pets':
  pets/migrations/0005_petrace.py
    + Create model PetRace


In [5]:
execute_from_command_line(['manage.py', 'migrate'])

Operations to perform:
  Apply all migrations: admin, auth, authtoken, contenttypes, core, pets, sessions
Running migrations:
  Applying pets.0005_petrace... OK


In [6]:
load_dotenv()
connection_string = os.getenv('DATABASE_URL')
connection_pool = pool.SimpleConnectionPool(1, 10,connection_string)
# Check if the pool was created successfully
if connection_pool:
    print("Connection pool created successfully")

Connection pool created successfully


In [11]:
dbCao = pd.read_excel(r"/home/errison/Documentos/AllPetSenior/Gdrive/Lista_de_racas.xlsx",sheet_name="cão")
dbCao['Porte'] = dbCao['Porte'].where(pd.notnull(dbCao['Porte']), None)
dbCao["Peso_minimo"] = dbCao["Peso_minimo"].replace(to_replace=",", value=".", regex=True)
dbCao["Peso_maximo"] = dbCao["Peso_maximo"].replace(to_replace=",", value=".", regex=True)
dbCao = dbCao.drop('Genero', axis=1)

dbGato = pd.read_excel(r"/home/errison/Documentos/AllPetSenior/Gdrive/Lista_de_racas.xlsx",sheet_name="gato")
dbGato["Peso_minimo"] = dbGato["Peso_minimo"].replace(to_replace=",", value=".", regex=True)
dbGato["Peso_maximo"] = dbGato["Peso_maximo"].replace(to_replace=",", value=".", regex=True)

In [12]:
dbCao

,Nome,Porte,Peso_minimo,Peso_maximo
0,Affenpinscher,None,4,6
1,Afghan Hound (Galgo afegão),None,22,27
2,Airedale Terrier,None,18,29
3,Akita americano,None,31,59
4,Akita Inu,None,31,54
...,...,...,...,...
411,Xoloitzcuintle,Pequeno,6.8,13.6
412,Xoloitzcuintle,Miniatura,4.5,6.8
413,Xoloitzcuintle,Médio,13.6,25
414,Yorkipoo,None,1.4,6.4


In [13]:
dbGato

,Nome,Peso_minimo,Peso_maximo
0,SRD,3,5
1,Abissínio,3,6
2,Bobtail Americano,3.5,6
3,American Curl,3.2,5.5
4,American Shorthair,2.7,6.8
...,...,...,...
75,Savannah,15,25
76,Nebelung,3,6
77,Selvagem,3,8
78,Ashera,12,15


In [14]:
for i in range(len(dbCao)):    
    pet_race = PetRace() 
    pet_race.name = dbCao['Nome'][i]
    pet_race.specie = 'DOG'
    pet_race.min_weight = dbCao['Peso_minimo'][i]
    pet_race.max_weight = dbCao['Peso_maximo'][i]
    pet_race.size = dbCao['Porte'][i]

    pet_race.save()

In [15]:
for i in range(len(dbGato)):    
    pet_race = PetRace() 
    pet_race.name = dbGato['Nome'][i]
    pet_race.specie = 'CAT'
    pet_race.min_weight = dbGato['Peso_minimo'][i]
    pet_race.max_weight = dbGato['Peso_maximo'][i]
    # pet_race.size = dbGato['Porte'][i]

    pet_race.save()

In [16]:
conn = connection_pool.getconn()
cur = conn.cursor()

cur.close()
connection_pool.putconn(conn)
connection_pool.closeall()    